# 1-Libreria RDFLib: Fundamentos

Estudiaremos la libreria rdflib basada en el lenguaje python, para la manipulación de metadatos en RDF.

In [1]:
# Instalaremos en nuestro entorno la siguiente libreria
!pip install rdflib

   ---------------------------------------- 0.0/565.1 kB ? eta -:--:--
   ---------------------------------------- 565.1/565.1 kB 6.6 MB/s eta 0:00:00


DEPRECATION: Loading egg at c:\users\cogito ergo sum\appdata\local\programs\python\python312\lib\site-packages\homework-0.1-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Ejemplo RDF 1

La interfaz principal que expone RDFLib para trabajar con RDF es **`Graph`**. Estos son contenedores no ordenados. Tienen varias operaciones básicas (por ejemplo, **add ()** para agregar un patron de tripleta) más métodos que buscan tripletas y las devuelve en orden arbitrario.
Los datos RDF son un gráfo donde los nodos son referencias URI (**URIRef**) o literales (**Literals**).

**Los Namespaces y Bindigs**. Para crear muchos `URIRefs` con el mismo espacio de nombre, es decir, URI con el mismo prefijo, RDFLib tiene la clase **rdflib.namespace.Namespace**.
En un grafo se puede enlazar un prefíjo a una URI de un namespace con el método: **rdflib.graph.bind()**.

En RDFLib existen namespace mas usados como es el RDF, FOAF y otros.

**FOAF** (Friend Of A Friend, literalmente "Amigo de un Amigo") es una ontología legible para las máquinas que describe a las personas, sus actividades y sus relaciones con otras personas y objetos




In [2]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, FOAF

# Crear un nuevo grafo RDF
g = Graph()

# Namespaces
EX = Namespace("http://example.org/animal#")
g.bind("ex", EX)
g.bind("foaf", FOAF)

# 🦁 Recurso URI: León
leon = URIRef(EX.Leon)
g.add((leon, RDF.type, EX.Mamifero))
g.add((leon, FOAF.name, Literal("Simba")))
g.add((leon, EX.edad, Literal(5)))
g.add((leon, EX.especie, Literal("Panthera leo")))

# Imprimir el número de "tripletas" en el Grafo
print(f"Grafo g tiene {len(g)} statements.")

# Imprimir el Grafo entero en el formato RDF Turtle
print(g.serialize(format="turtle"))

Grafo g tiene 4 statements.
@prefix ex: <http://example.org/animal#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Leon a ex:Mamifero ;
    ex:edad 5 ;
    ex:especie "Panthera leo" ;
    foaf:name "Simba" .




### Ejemplo RDF 2

Es importante recordar que otro tipo de nodo en un grafo son los nodos blancos (**BNode**).

In [3]:
from rdflib import BNode

# 🐦 Nodo en blanco: Pájaro sin nombre
unknown = BNode()
g.add((unknown, RDF.type, EX.Ave))
g.add((unknown, FOAF.name, Literal("Pájaro sin nombre")))
g.add((unknown, EX.color, Literal("Amarillo")))

# Imprimir el número de "tripletas" en el Grafo
print(f"Grafo g tiene {len(g)} statements.")

# Imprimir el Grafo entero en el formato RDF Turtle
print(g.serialize(format="turtle"))


Grafo g tiene 7 statements.
@prefix ex: <http://example.org/animal#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Leon a ex:Mamifero ;
    ex:edad 5 ;
    ex:especie "Panthera leo" ;
    foaf:name "Simba" .

[] a ex:Ave ;
    ex:color "Amarillo" ;
    foaf:name "Pájaro sin nombre" .




### Ejemplo RDF 3

Es posible recorrer cada una de las tripletas del grafo e identificar que tipo de elemento es el sujeto (s), el predicado (p) y el objeto (o).

In [4]:
# Imprimir el número de "tripletas" en el Grafo
print(f"Grafo g tiene {len(g)} statements.")

# Podemos recorrer las tripletas de la siguiente manera
for s, p, o in g:
  res = ""

  # Clasfico los valores segun su tipo
  if (type(s) is URIRef):
    res += "URI"
  elif (type(s) is BNode):
    res += "blank"

  if (type(p) is URIRef):
    res += " URI "

  if (type(o) is URIRef):
    res += "URI"
  elif (type(o) is Literal):
    res += "literal"
  elif (type(o) is BNode):
    res += "blank"

  print(res)
  print(s, p, o)

Grafo g tiene 7 statements.
URI URI URI
http://example.org/animal#Leon http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://example.org/animal#Mamifero
blank URI literal
N6ab62fc468244cd6b981ca1dfdc3ffb6 http://xmlns.com/foaf/0.1/name Pájaro sin nombre
URI URI literal
http://example.org/animal#Leon http://xmlns.com/foaf/0.1/name Simba
URI URI literal
http://example.org/animal#Leon http://example.org/animal#edad 5
blank URI URI
N6ab62fc468244cd6b981ca1dfdc3ffb6 http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://example.org/animal#Ave
blank URI literal
N6ab62fc468244cd6b981ca1dfdc3ffb6 http://example.org/animal#color Amarillo
URI URI literal
http://example.org/animal#Leon http://example.org/animal#especie Panthera leo


### Ejemplo RDF 4

Un elemento que puede hacer parte de un grafo son las listas. En este caso se estudia la creación de una lista tipo **colección** (compuesta por cabeza y cola) la cual no es editable.

Otro aspecto importante es que en el caso de los Literales pueden asociarsele un tipo de dato haciendo uso del XMLSchema. En este caso el namespace estandar de este es XSD, el cual contiene el uri de la respectiva especificación. Este es un vocabulario que esta implementado en el RDFLib.

Por ultimo es posible incluir para los Literales incluirle su lang (tipo de idioma).

In [5]:
from rdflib.namespace import XSD
from rdflib.collection import Collection

# 🐘 Recurso URI: Elefante
elephant = URIRef(EX.Elefante)
g.add((elephant, RDF.type, EX.Mamifero))
g.add((elephant, FOAF.name, Literal("Dumbo", lang="es")))  # 👈 Literal con lenguaje español ("es")
g.add((elephant, EX.edad, Literal(10)))
g.add((elephant, EX.numeroPatas, Literal(4, datatype=XSD.integer)))  # 👈 Literal con tipo int


# 🍃 Lista RDF (Colección): Alimentos favoritos del elefante
favorite_foods = BNode()
Collection(g, favorite_foods, [
    Literal("Hojas"),
    Literal("Frutas"),
    Literal("Caña")
])
g.add((elephant, EX.alimentosFavoritos, favorite_foods))

# Imprimir el Grafo entero en el formato RDF Turtle
print(g.serialize(format="turtle"))

# Guardar el grafo en un archivo Turtle
g.serialize("animales.rdf", format="turtle")

@prefix ex: <http://example.org/animal#> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Elefante a ex:Mamifero ;
    ex:alimentosFavoritos ( "Hojas" "Frutas" "Caña" ) ;
    ex:edad 10 ;
    ex:numeroPatas 4 ;
    foaf:name "Dumbo"@es .

ex:Leon a ex:Mamifero ;
    ex:edad 5 ;
    ex:especie "Panthera leo" ;
    foaf:name "Simba" .

[] a ex:Ave ;
    ex:color "Amarillo" ;
    foaf:name "Pájaro sin nombre" .




<Graph identifier=N2a9b4d9aee03498cbe7bc7d2cb13a15b (<class 'rdflib.graph.Graph'>)>

### Ejemplo RDF 5

Profundizando en el uso de namespace, rdflib tiene algunas ontologias conocidad que atraves de sus **namespaces** se pueden **importar** directamente.


In [6]:
# Importamos rdflib
from rdflib import Graph, Literal, URIRef, Namespace

# rdflib tiene algunas ontologías y usando sus namespaces se pueden importar
from rdflib.namespace import FOAF, SKOS, DC, RDF

# Creamos un grafo vacio
g = Graph()

# Creamos un namespace
n = Namespace("http://example.org/")

# Creamos algunos recursos
n.aliciabarberis
n.bob
n.bookExample
n.book

# Vocabulario FOAF
g.add((n.aliciabarberis, RDF.type, FOAF.Person))
g.add((n.aliciabarberis, FOAF.name, Literal("Alicia")))
g.add((n.aliciabarberis, FOAF.mbox, URIRef("mailto:alicia-barberis@example.org")))
g.add((n.aliciabarberis, FOAF.knows, n.bob))

# Vocabulario Dublin core
g.add((n.bookExample, DC.title, Literal("El infierno de los vivos")))
g.add((n.bookExample, DC.creator, n.aliciabarberis))
g.add((n.bookExample, DC.date, Literal("2005-12-07")))
g.add((n.bookExample, DC.publisher, n.bob))


# Vocabulario SKOS core
g.add((n.book, RDF.type, SKOS.Concept))
g.add((n.book, SKOS.prefLabel, Literal("Libro")))
g.add((n.book, SKOS.altLabel, Literal("Ejemplar")))
g.add((n.book, SKOS.definition, Literal("Conjunto de hojas de papel, normalmente encuadernadas")))

# Podemos recorrer e imprimir las tripletas de la siguiente manera
for s, p, o in g:
    print(s, p, o)


http://example.org/aliciabarberis http://xmlns.com/foaf/0.1/name Alicia
http://example.org/book http://www.w3.org/2004/02/skos/core#prefLabel Libro
http://example.org/book http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2004/02/skos/core#Concept
http://example.org/book http://www.w3.org/2004/02/skos/core#altLabel Ejemplar
http://example.org/book http://www.w3.org/2004/02/skos/core#definition Conjunto de hojas de papel, normalmente encuadernadas
http://example.org/bookExample http://purl.org/dc/elements/1.1/title El infierno de los vivos
http://example.org/bookExample http://purl.org/dc/elements/1.1/publisher http://example.org/bob
http://example.org/aliciabarberis http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Person
http://example.org/bookExample http://purl.org/dc/elements/1.1/creator http://example.org/aliciabarberis
http://example.org/bookExample http://purl.org/dc/elements/1.1/date 2005-12-07
http://example.org/aliciabarberis http://xml

### Ejemplo RDF 5

A partir del ejemplo anterior, estudiamos como poder consultar un grafo con RDFLib al evaluar un componente específico recorriendo las tripletas mediante un **for** y filtrandolas mediante un **if**.

In [7]:
# Podemos recorrer las tripletas de la siguiente manera
for s, p, o in g:

  # filtrando las tripletas por una condicion
  if ("http://xmlns.com/foaf/0.1/name" == str(p)):
    print(s, o)

http://example.org/aliciabarberis Alicia


### Ejemplo RDF 6

Se puede **modificar** el valor de una propiedad mediante el **método set**. Igualmente se puede **eliminar tripletas** con **remove**, indicando la tripleta específica o usando **None** para dejar
sin especificar algún elemento.

In [8]:
# Importar rdflib y componentes RDF
import rdflib
from rdflib import Graph, Literal, BNode, Namespace, RDF, FOAF

# Creamos un grafo vacio y definimos un namespace
g = Graph()
mm = Namespace('http://mundo.mundial.org/persona/')

# Crear un recurso (RDF URI node) y sus tripletas asociadas
pedro = mm.pedro
maria = mm.maria

g.add((pedro, RDF.type, FOAF.Person))
g.add((pedro, FOAF.name, Literal('Pedro')))
g.add((pedro, FOAF.age, Literal(22)))

g.add((maria, RDF.type, FOAF.Person))
g.add((maria, FOAF.name, Literal('Maria')))
g.add((maria, FOAF.age, Literal(25)))

g.add((pedro, FOAF.knows, maria))

# Imprimir el número de "tripletas" en el Grafo
print(f"Grafo g tiene inicialmente {len(g)} statements.")

# Recorrer e imprimir las tripletas
for s, p, o in g:
    print(s, p, o)

# Modificar el valor de una propiedad con el método set
g.set((pedro, FOAF.age, Literal(23)))

# Eliminar la tripleta pedro conoce a maria
g.remove((pedro, FOAF.knows, maria))

# Eliminar todas las tripletas que se refieren a maria usando None
g.remove((maria, None, None))

# Imprimir el número de "tripletas" en el Grafo
print(f"Grafo g tiene finalmente {len(g)} statements.")

# Recorrer e imprimir las tripletas
for s, p, o in g:
    print(s, p, o)



Grafo g tiene inicialmente 7 statements.
http://mundo.mundial.org/persona/pedro http://xmlns.com/foaf/0.1/knows http://mundo.mundial.org/persona/maria
http://mundo.mundial.org/persona/maria http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Person
http://mundo.mundial.org/persona/maria http://xmlns.com/foaf/0.1/name Maria
http://mundo.mundial.org/persona/pedro http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Person
http://mundo.mundial.org/persona/pedro http://xmlns.com/foaf/0.1/name Pedro
http://mundo.mundial.org/persona/pedro http://xmlns.com/foaf/0.1/age 22
http://mundo.mundial.org/persona/maria http://xmlns.com/foaf/0.1/age 25
Grafo g tiene finalmente 3 statements.
http://mundo.mundial.org/persona/pedro http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://xmlns.com/foaf/0.1/Person
http://mundo.mundial.org/persona/pedro http://xmlns.com/foaf/0.1/name Pedro
http://mundo.mundial.org/persona/pedro http://xmlns.com/foaf/0.1/age 23


### Ejemplo RDF 7

Para cargar un archivo se debe saber el formato (turtle,xml, n3, ...) y utilizar el
método **parse** del objeto Graph. Este ejemplo carga un archivo local.

In [9]:
# Importamos rdflib
import rdflib

# Cargamos el grafo en formato turtle
g.parse("/content/data.ttl", format="ttl")


# Guardaremos la ontologia en formato xml
g.serialize(destination='outputio1.xml', format='xml')

# format puede tener los siguientes valores https://rdflib.readthedocs.io/en/stable/plugin_serializers.html

FileNotFoundError: [Errno 2] No such file or directory: '\\content\\data.ttl'

### Ejemplo RDF 8


Se puede usar una URL para cargar archivos remotos

In [ ]:
# Importamos rdflib
import rdflib
from rdflib import Literal, BNode, URIRef

# Creamos un grafo vacio
g = rdflib.Graph()

# Cargar un grafo remotamente desde una url (al no indicarle el formato, lo identifica automáticamente)
g.parse("http://www.w3.org/People/Berners-Lee/card")


# Guardaremos la ontologia en formato xml
g.serialize(destination='outputio2.ttl', format='ttl')

### Ejemplo RDF 9

El operador **in** sirve para verificar si un triple (o también un nodo) existe dentro de un grafo RDF.

El operador **in** esta sobrecargado para los grafos, asi que podemos hacer consultas sencillas de existencia (Caso1) al igual que se puede usar también el método **triples** para hacer una consulta simple, usando **None** en la parte de la tripleta que se quiere que sea variable (caso2, caso3).

In [ ]:
import rdflib
from rdflib import Graph, Literal, BNode, Namespace, FOAF

g = Graph()
n = Namespace('http://profesores.edu.co/')

p1 = n.Jaime
p2 = n.Alejandro
v = Literal(40)
g.add((p1, FOAF.age, v))
g.add((p1, FOAF.name, Literal("Jaime")))
g.add((p2, FOAF.name, Literal("Alejandro")))
g.add((p2, FOAF.age, Literal(20)))

# Caso1: Consulta simple
print("Caso1")
if (p1, FOAF.age, v) in g:
  print ("Jaime tiene edad")

# Caso2: Consulta usando método triples y None
print("Caso2")
for s, p, o in g.triples((None, FOAF.age, None)):
  print(s, p, o)

# Caso3: Consulta usando método triples y None
print("Caso3")
for s, p, o in g.triples((p1, None, None)):
  print(s, p, o)
